In [2]:


%matplotlib inline
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
from sklearn import preprocessing

# load data
train_data = pd.read_csv("train.csv")
train_data.head()

train_data.shape

train_data.duplicated(subset='Id').sum()

train_data.describe()

train_data.dtypes.value_counts()



test_data = pd.read_csv("test.csv")
test_data.head()

test_data.shape

test_data.duplicated(subset='Id').sum()

test_data.describe()

test_data.dtypes.value_counts()

# check for missing data in columns
train_data.isnull().sum().sort_values(ascending=False).head(10) / len(train_data) *100

test_data.isnull().sum().sort_values(ascending=False).head(10) / len(train_data) *100

# we drop irrelivant columns
train_data.drop(["PoolQC", "MiscFeature","Alley","Fence","FireplaceQu"], axis = 1, inplace = True)

train_data.head()

test_data.drop(["PoolQC", "MiscFeature","Alley","Fence","FireplaceQu"], axis = 1, inplace = True)

test_data.head()

#encoding category features
oe = preprocessing.OrdinalEncoder()

# Train Categories Encoding
train_categorical_features = train_data.select_dtypes(include=['object']).columns
train_numerical_features= train_data.select_dtypes([np.int64,np.float64]).columns

train_data[train_categorical_features]=oe.fit_transform(train_data[train_categorical_features])

# Test Categories Encoding
test_categorical_features = test_data.select_dtypes(include=['object']).columns
test_numerical_features= test_data.select_dtypes([np.int64,np.float64]).columns

test_data[test_categorical_features]=oe.fit_transform(test_data[test_categorical_features])

test_data.isnull().sum().sort_values(ascending=False).head(15) / len(test_data) *100

train_data[train_numerical_features]=train_data[train_numerical_features].fillna(train_data[train_numerical_features].median())

for feature in train_categorical_features:
    train_data[feature].interpolate(method ='linear', limit_direction ='forward', inplace=True)
    train_data[feature].interpolate(method ='linear', limit_direction ='backward',inplace=True)
train_data.isnull().sum().sort_values(ascending=False).head(10) / len(train_data) *100

test_data[test_numerical_features]=test_data[test_numerical_features].fillna(test_data[test_numerical_features].median())

for feature in test_categorical_features:
    test_data[feature].interpolate(method ='linear', limit_direction ='forward', inplace=True)
    test_data[feature].interpolate(method ='linear', limit_direction ='backward',inplace=True)

test_data.isnull().sum().sort_values(ascending=False).head(15) / len(test_data) *100

corr_with=train_data.corrwith(train_data["SalePrice"]).sort_values(ascending=False)
print(corr_with[corr_with>0].head(20),"\n-------------------------")

print(corr_with[corr_with<0].tail(20))

list_of_columns =["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF",
                  "1stFlrSF","FullBath","TotRmsAbvGrd","YearBuilt","YearRemodAdd",
                  "MasVnrArea","GarageYrBlt","Fireplaces","BsmtFinSF1","Foundation",
                  "LotFrontage","WoodDeckSF","2ndFlrSF","OpenPorchSF",
                  "ExterQual","BsmtQual","KitchenQual","GarageFinish","GarageType","HeatingQC"]


X = train_data[list_of_columns]
y = train_data.SalePrice
corr = X.corr()
sns.set(rc = {'figure.figsize':(16,12)})
sns.heatmap(corr,mask=((np.triu(np.ones_like(corr)))),cmap="YlGnBu",annot=True)

list_of_columns =["OverallQual","GrLivArea","GarageCars","TotalBsmtSF","1stFlrSF",
                  "FullBath","YearBuilt","YearRemodAdd","BsmtFinSF1","Foundation",
                  "LotFrontage","WoodDeckSF","MasVnrArea","Fireplaces",
                  "ExterQual","BsmtQual","KitchenQual","GarageFinish","GarageType","HeatingQC"]

X = train_data[list_of_columns]
y = train_data.SalePrice
corr = X.corr()
sns.set(rc = {'figure.figsize':(16,12)})
sns.heatmap(corr,mask=((np.triu(np.ones_like(corr)))),cmap="YlGnBu",annot=True)

scalar =preprocessing.StandardScaler()
X_scaled=pd.DataFrame(scalar.fit_transform(X),columns = X.columns)
y = train_data.SalePrice               
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y,
                                                  test_size=0.08,
                                                 random_state = 42)
print('train:', len(X_train), 'validation:', len(X_val))

# Linear Regression

def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

def export_Submission_file(predicition_dataframe,output_file_name):
    Output = pd.DataFrame()
    Output['Id'] = predicition_dataframe['Id']
    Output['SalePrice'] = predicition_dataframe['y_pred']
    Output.to_csv("../"+output_file_name+".txt", index=False)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred=model.predict(X_train)
y= y_train.ravel()
y_pred=y_pred.ravel()
y_pred =abs(y_pred)

print("Root-Mean-Squared-Error :",compute_rmsle(y,y_pred,4))
print("R2 :",metrics.r2_score(y_train,y_pred))













ModuleNotFoundError: No module named 'matplotlib'